In [1]:
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.2/145.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 12.6 MB/s eta 0:00:00


In [2]:
# Install required libraries
!pip install langchain-community
!pip install ipywidgets
!pip install langchain pandas huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.23
    Uninstalling langchain-0.3.23:
      Successfully uninstalled langchain-0.3.23
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.1 MB/s eta 0:00:00


In [3]:
!pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.7 MB/s eta 0:00:00


In [4]:
# Import libraries
import sqlite3
import pandas as pd
from langchain_community.utilities import SQLDatabase
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import HuggingFaceHub
from langgraph.graph import StateGraph, END
from langchain import hub
import os
import requests

In [8]:
# Function to load northwind.db into in-memory SQLite
# Working with northwind.db
def load_db_to_memory(db_file_path):
    try:
        # Connect to the on-disk database
        disk_conn = sqlite3.connect(db_file_path)
        # Connect to in-memory database
        mem_conn = sqlite3.connect(":memory:")

        # Copy the database to memory
        disk_conn.backup(mem_conn)

        # Close the on-disk connection
        disk_conn.close()

        print("Successfully loaded northwind.db into in-memory SQLite")
        return mem_conn
    except Exception as e:
        print(f"Error loading northwind.db: {e}")
        raise

# Option 2: Download northwind.db from a URL (uncomment if applicable)
# Load the database into memory
conn = load_db_to_memory('northwind.db')
cursor = conn.cursor()

Successfully loaded northwind.db into in-memory SQLite


In [9]:
# List all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('Categories',), ('sqlite_sequence',), ('CustomerCustomerDemo',), ('CustomerDemographics',), ('Customers',), ('Employees',), ('EmployeeTerritories',), ('Order Details',), ('Orders',), ('Products',), ('Regions',), ('Shippers',), ('Suppliers',), ('Territories',)]


In [10]:
# List all tables another version for educational pupose and is working
db = SQLDatabase.from_uri("sqlite:///northwind.db")
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['Categories', 'CustomerCustomerDemo', 'CustomerDemographics', 'Customers', 'EmployeeTerritories', 'Employees', 'Order Details', 'Orders', 'Products', 'Regions', 'Shippers', 'Suppliers', 'Territories']


In [11]:
# Verify data loaded
for row in cursor.execute("SELECT * FROM employees LIMIT 5"):
    print(row)

# Optional: get table schema
cursor.execute("PRAGMA table_info(employees)")
print(cursor.fetchall())

for row in cursor.execute("SELECT * FROM 'order details' LIMIT 5"):
    print(row)
cursor.execute("PRAGMA table_info('order details')")
print(cursor.fetchall())

for row in cursor.execute("SELECT * FROM orders LIMIT 5"):
    print(row)
cursor.execute("PRAGMA table_info(orders)")
print(cursor.fetchall())

for row in cursor.execute("SELECT * FROM products LIMIT 5"):
    print(row)
cursor.execute("PRAGMA table_info(products)")
print(cursor.fetchall())

(1, 'Davolio', 'Nancy', 'Sales Representative', 'Ms.', '1968-12-08', '2012-05-01', '507 - 20th Ave. E.Apt. 2A', 'Seattle', 'North America', '98122', 'USA', '(206) 555-9857', '5467', b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00\x00d\x00d\x00\x00\xff\xec\x00\x11Ducky\x00\x01\x00\x04\x00\x00\x00N\x00\x00\xff\xee\x00&Adobe\x00d\xc0\x00\x00\x00\x01\x03\x00\x15\x04\x03\x06\n\r\x00\x00\r\xc7\x00\x00\x14\xf8\x00\x00\x1f\xc7\x00\x000\x18\xff\xdb\x00\x84\x00\x03\x02\x02\x02\x02\x02\x03\x02\x02\x03\x04\x03\x02\x03\x04\x05\x03\x03\x03\x03\x05\x05\x04\x04\x05\x04\x04\x05\x07\x05\x06\x06\x06\x06\x05\x07\x07\x08\x08\t\x08\x08\x07\x0b\x0b\x0c\x0c\x0b\x0b\x0c\x0c\x0c\x0c\x0c\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x01\x03\x03\x03\x05\x05\x05\n\x07\x07\n\x0f\x0c\n\x0c\x0f\x12\x0e\x0e\x0e\x0e\x12\x11\x0e\x0e\x0e\x0e\x0e\x11\x11\x0e\x0e\x0e\x0e\x0e\x0e\x11\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\xff\xc2\x00\x11\x08\x00\xaa\x00\xa0\x

In [12]:
from langchain_openai import ChatOpenAI  # Use ChatOpenAI instead of OpenAI
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
# Initialize LLM (OpenAI gpt-4o-mini)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# response = llm.invoke("Tell me a joke about data scientists.")    # Debug message
# print(response)                                                   # Debug message


In [13]:
# Set up LangChain SQL toolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()

# print(tools)                                                      # Debug message

from langchain_core.prompts import PromptTemplate
prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")
prompt_template.messages[0].pretty_print()

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


================================ System Message ================================

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to th

In [15]:
from langchain.agents import create_sql_agent
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

# Step 4: Run a query
agent_executor.invoke({"input": "List the total sales by region"})



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  Categories, CustomerCustomerDemo, CustomerDemographics, Customers, EmployeeTerritories, Employees, Order Details, Orders, Products, Regions, Shippers, Suppliers, TerritoriesI need to find the relevant tables that contain sales and region information. The "Orders" table likely contains sales data, and the "Regions" table may provide regional information. I will check the schema of both tables to identify the relevant columns for my query.

Action: sql_db_schema  
Action Input: Orders, Regions  
CREATE TABLE "Orders" (
	"OrderID" INTEGER NOT NULL, 
	"CustomerID" TEXT, 
	"EmployeeID" INTEGER, 
	"OrderDate" DATETIME, 
	"RequiredDate" DATETIME, 
	"ShippedDate" DATETIME, 
	"ShipVia" INTEGER, 
	"Freight" NUMERIC DEFAULT 0, 
	"ShipName" TEXT, 
	"ShipAddress" TEXT, 
	"ShipCity" TEXT, 
	"ShipRegion" TEXT, 
	"ShipPostalCode" TEXT, 
	"ShipCountry" TEXT, 
	PRIMARY KEY ("OrderID"), 
	FOREIGN KEY("ShipVia") RE

{'input': 'List the total sales by region',
 'output': 'The total sales by region are: \n- Western Europe: $1,247,426.25\n- South America: $733,001.50\n- North America: $705,528.00\n- Southern Europe: $396,037.25\n- British Isles: $366,536.00\n- Central America: $228,138.75\n- Northern Europe: $186,604.00\n- Scandinavia: $130,529.50\n- Eastern Europe: $53,669.25'}

In [16]:
conn.close()